# BQML Імпорт моделі Scikit-Learn за допомогою ONNX

У цьому блокноті показано, як імпортувати модель/конвеєр [scikit-learn](https://scikit-learn.org/stable/) у BigQuery ML для прогнозування безпосередньо всередині BigQuery за допомогою функції [ML.PREDICT()](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-predict).  Це досягається шляхом перетворення моделі/конвеєра `scikit-learn` у формат [ONNX](https://onnx.ai/) - відкритий стандарт для інтероперабельності машинного навчання - і подальшого імпорту його безпосередньо в BigQuery ML.

$$\textrm{scikit-learn} \Longrightarrow \textrm{ONNX} \Longrightarrow \textrm{BigQuery ML}$$

**BigQuery ML Inference Engine**

За допомогою BigQuery ML ви можете [імпортувати моделі, навчені поза BigQuery](https://cloud.google.com/bigquery/docs/inference-overview#inference_using_imported_models) у форматах [TensorFlow](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-tensorflow), [TensorFlow Lite](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-tflite), [XGBoost](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-xgboost) та [ONNX](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-onnx). Це частина того, що називається BigQuery ML [Inference Engine](https://cloud.google.com/bigquery/docs/inference-overview#inference_using_imported_models), яка має методи для роботи з моделями, навченими/розміщеними поза BigQuery ML, використовуючи для зручності той самий SQL API.  

Прочитайте статтю в блозі, яка анонсує появу рушія виводу з березня 2023 року [тут](https://cloud.google.com/blog/products/data-analytics/introducing-bigquery-ml-inference-engine), щоб отримати детальний огляд.

**ONNX - Open Neural Network Exchange**

Формат [ONNX](https://onnx.ai/) є відкритим стандартом для інтероперабельності машинного навчання.  Це робить моделі придатними для використання в багатьох фреймворках, інструментах і середовищах виконання.  
- [підтримувані фреймворки](https://onnx.ai/supported-tools.html#buildModel)
- та багато іншого за допомогою [onnxmltools](https://github.com/onnx/onnxmltools)

**Перетворення scikit-learn на ONNX**

Хоча [onnxmltools](https://github.com/onnx/onnxmltools) має обгортку для [skl2onnx](https://github.com/onnx/sklearn-onnx/), у цьому посібнику показано використання безпосередньо пакету `skl2onnx`.


**ONNX за допомогою ONNXRuntime**

Пакет [onnxruntim](https://onnxruntime.ai/) можна використовувати для прогнозування/виведення моделі у форматі ONNX.  Він демонструється у цьому ноутбуці, але не є обов'язковим для BigQuery ML, який надає повністю кероване виконання з моделлю, тому користувачам не потрібно налаштовувати цю частину.

**Передумови**

У цьому ноутбуці використовується модель `scikit-learn`, побудована заздалегідь і збережена в GCS у вигляді pickle-файлу (`.pkl`).

**Ресурси**
- Підручник [Make predictions with scikit-learn models in ONNX format](https://cloud.google.com/bigquery/docs/making-predictions-with-sklearn-models-in-onnx-format)

In [ ]:
PROJECT_ID = 'fit-cloud-course' # замінити на ідентифікатор проекту

In [ ]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Налаштування

pip:

In [ ]:
try:
    import skl2onnx
except ImportError:
    !pip install --user skl2onnx onnxruntime  -U -q

вхідні дані:

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

In [ ]:
REGION = 'us-central1'
EXPERIMENT = 'import-onnx-sklearn'
SERIES = 'bqml'

# вихідні дані
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Навчання моделі
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id'

пакети:

In [ ]:
from datetime import datetime

from google.cloud import aiplatform
from google.cloud import bigquery
from google.cloud import storage

import numpy as np
import sklearn
import pickle

import skl2onnx
import onnxruntime

клієнти:

In [ ]:
aiplatform.init(project = PROJECT_ID, location = REGION)
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client()

---
## Отримання даних для прогнозування

In [ ]:
n = 10
pred = bq.query(
    query = f"""
        SELECT * EXCEPT({VAR_TARGET}, {VAR_OMIT}, splits)
        FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}
        WHERE splits='TEST'
        LIMIT {n}
        """
).to_dataframe()

In [ ]:
pred.head()

Форма як екземпляри: словники пар ключ:значення тільки для ознак, що використовуються в моделі

In [ ]:
newobs = pred.to_dict(orient='records')
#newobs[0]

In [ ]:
len(newobs)

In [ ]:
newobs[0]

---

## Отримати модель для прогнозування

Ця секція отримує поточну активну модель, яка використовується на кінцевій точці прогнозування Vertex AI.

>Якщо ви вже знаєте розташування файлів вашої моделі у GCS Bucket, то цю секцію можна оминути, зберігши розташування моделі за допомогою: `model_uri = 'gs://bucket/path/to/files'.

In [ ]:
# Серія 04 створює моделі на основі scikit-learn
PREVIOUS_SERIES = '04'

### Отримати кінцеву точку

Посилання: [aiplatform.Endpoint](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Endpoint)

In [ ]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={PREVIOUS_SERIES}")
endpoint = endpoints[0]

In [ ]:
print(f'Перегляньте кінцеву точку в консолі:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

In [ ]:
[list(newobs[0].values())]

In [ ]:
endpoint.predict(instances = [list(newobs[0].values())]).predictions

### Переглянути інформацію про модель

Посилання: [aiplatform.Model](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model)

In [ ]:
vertex_model = aiplatform.Model(
    model_name = endpoint.list_models()[0].model + f'@{endpoint.list_models()[0].model_version_id}'
)

In [ ]:
vertex_model.display_name

In [ ]:
vertex_model.version_id

In [ ]:
vertex_model.name

In [ ]:
vertex_model.uri

In [ ]:
!gsutil ls {vertex_model.uri}

In [ ]:
bucket = gcs.bucket(PROJECT_ID)
for blob in bucket.list_blobs(prefix = vertex_model.uri.split(f'gs://{PROJECT_ID}/')[1]):
    print(blob.name)
    if blob.name.split('.pkl')[-1] == '.pkl': break;

In [ ]:
print(f'Review the model in the Vertex AI Model Registry:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{vertex_model.name}/versions/{vertex_model.version_id}/properties?project={PROJECT_ID}')

---
## Локальна модель: scikit-learn

Завантажуємо модель у блокнот.

In [ ]:
pickle_in = blob.download_as_string()
local_model = pickle.loads(pickle_in)

Отримуйте локальні прогнози за допомогою моделі:

In [ ]:
[list(newobs[0].values())]

In [ ]:
local_model.predict([list(newobs[0].values())])

In [ ]:
local_model.predict_proba([list(newobs[0].values())])

---
## Перетворення моделі: ONNX

- За допомогою [sklearn-onnx](https://onnx.ai/sklearn-onnx/)
    - Всі доступні типи даних у [джерелі](https://github.com/onnx/sklearn-onnx/blob/main/skl2onnx/common/data_types.py)
    - більше про опцію zipmap [тут](https://onnx.ai/sklearn-onnx/auto_tutorial/plot_dbegin_options_zipmap.html)

In [ ]:
initial_types = []
for feature in preds.dtypes.apply(lambda x: x.name).to_dict().items():
    if feature[1] == 'Int64': tensor_type = skl2onnx.common.data_types.Int64TensorType([None, 1])
    elif feature[1] == 'float64': tensor_type = skl2onnx.common.data_types.FloatTensorType([None, 1])
    # інші типи даних тут за потреби
    initial_types.append((feature[0], tensor_type))

In [ ]:
onnx_model = skl2onnx.convert_sklearn(local_model, initial_types = initial_types, options = {id(local_model): {'zipmap': False}})

## Локальний тест моделі ONNX

- За допомогою [onnxruntime](https://onnxruntime.ai/)

In [ ]:
local_onnx = onnxruntime.InferenceSession(onnx_model.SerializeToString())

In [ ]:
test_ob = newobs[0].copy()
for v in test_ob:
    if type(test_ob[v]) == int:
        test_ob[v] = np.array([[test_ob[v]]], dtype = np.int64)
    elif type(test_ob[v]) == float:
        test_ob[v] = np.array([[test_ob[v]]], dtype = np.float32)
test_ob

In [ ]:
local_onnx.run(None, test_ob)

In [ ]:
local_onnx.run(None, test_ob)[0]

---
## Імпорт моделі BigQuery ML

Довідка: [The CREATE MODEL statement for importing ONNX models](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-onnx)

Збережіть модель ONNX в GCS:

In [ ]:
blob = bucket.blob(f'{SERIES}/{EXPERIMENT}/model.onnx')
blob.upload_from_string(onnx_model.SerializeToString())

Створення ML-моделі BigQuery:

In [ ]:
query = f"""
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.{SERIES}-{EXPERIMENT}`
    OPTIONS(
        MODEL_TYPE = 'ONNX',
        MODEL_PATH = 'gs://{PROJECT_ID}/{SERIES}/{EXPERIMENT}/*'
    )
"""
print(query)

In [ ]:
job = bq.query(query = query)
job.result()
(job.ended-job.started).total_seconds()

## Прогнозування з BigQuery ML: ML.PREDICT

In [ ]:
query = f"""
SELECT *
FROM ML.PREDICT (MODEL `{BQ_PROJECT}.{BQ_DATASET}.{SERIES}-{EXPERIMENT}`,(
    SELECT * 
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
    WHERE splits = 'TEST'
    LIMIT 1)
  )
"""
pred = bq.query(query = query).to_dataframe()

In [ ]:
pred

In [ ]:
print(query)